In [1]:
import pandas as pd

data_hh = pd.read_csv('data/dst-3.0_16_1_hh_database.csv', sep=';')

def get_city_category(location_data:str):
    million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск',
                  'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград']
    msk_spb = ['Москва', 'Санкт-Петербург']
    city = location_data.split(' , ')[0]
    million_city = 'город-миллионник'
    other_cities = 'другие'
    
    if city in msk_spb:
        return city
    elif city in million_cities:
        return million_city
    else:
        return other_cities

def is_ready_to_relocate(location_data:str):
    ready_to_relocate = False
    location_data_list = location_data.split(' , ')
    metro_indicator = 'м. '
    
    if metro_indicator in location_data_list[1]:
        relocation = location_data_list[2]
    else:
        relocation = location_data_list[1]
        
    relocation = relocation.split(' (')[0] # delete extra info in brackets about relocation
    first_word = relocation.split()[0]
    
    if first_word != 'не':
        ready_to_relocate = True
    
    return ready_to_relocate

def is_ready_to_business_trip(location_data:str):
    ready_to_business_trip = False
    total_length = len(location_data.split(' , '))
    correct_length = 3 # [0] - City, [1] - relocation, [2] - business trip
    business_trip_info = location_data.split(' , ')[-1]
    first_word = business_trip_info.split()[0]
    
    if first_word != 'не' and total_length == correct_length: # if business trip data has been lost
        ready_to_business_trip = True
    
    return ready_to_business_trip
    
data_hh['Город'] = data_hh['Город, переезд, командировки'].apply(get_city_category)
data_hh['Готовность к переезду'] = data_hh['Город, переезд, командировки'].apply(is_ready_to_relocate)
data_hh['Готовность к командировкам'] = data_hh['Город, переезд, командировки'].apply(is_ready_to_business_trip)
applicants_by_cities = data_hh['Город'].value_counts(normalize=True).apply(lambda x: str(round(x * 100)) + '%')
relocate_and_trip = str(round(data_hh[(data_hh['Готовность к переезду'] & data_hh['Готовность к командировкам'])].shape[0] / data_hh.shape[0] * 100)) + '%'
display('Процент соискателей по городам:',
        applicants_by_cities,
        'Процент соискателей, готовых одновременно к переездам и командировкам:',
        relocate_and_trip)

'Процент соискателей по городам:'

Москва              37%
другие              35%
город-миллионник    16%
Санкт-Петербург     11%
Name: Город, dtype: object

'Процент соискателей, готовых одновременно к переездам и командировкам:'

'21%'